# Python 객체를 통해 DBMS로 DB관리

Python(execute*) -> DBMS -> DB   
1. Connection 생성  
2. Cursor를 부여받아 작업  

  
  
<br/>
    
# SQLite : local file에 나 혼자 작업하는 DB

## 1. SQL 가장 간단한 예제

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [6]:
type(con), type(cur)

(sqlite3.Connection, sqlite3.Cursor)

In [8]:
# TABLE 생성
cur.execute('''
    CREATE TABLE CITY(
        CNO   INTEGER PRIMARY KEY,
        CNAME TEXT
    );
''')

# TABLE 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[]

In [12]:
# data 삽입
cur.execute('''
    INSERT INTO CITY(CNO, CNAME) VALUES(NULL, '강북구')
''')

# TABLE 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구'), (2, '동대문구'), (3, '강동구'), (4, '강북구')]

In [13]:
# 가장 마지막에 추가된 row id
cur.lastrowid

4

In [14]:
# DB 연결 종료
con.close()

<br/>

## 2. SQL 파라미터 전달 방법(qmark, named style) 이용하여 DB 생성

In [17]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [18]:
# DB 생성
cur.execute('''
    CREATE TABLE CITY(
        CNO   INTEGER PRIMARY KEY,
        CNAME TEXT
    )
''')

# 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[]

In [19]:
# 데이터 삽입1 : 기본적인 3가지 방법

# 변수 명시
cur.execute('''
    INSERT INTO CITY(CNO, CNAME) VALUES(1, '강동구')
''')

# pk인 CNO는 지정 안해도 자동으로 들어감
cur.execute('''
    INSERT INTO CITY(CNAME) VALUES('동대문구')
''')

# pk 자리에 NULL 집어넣어도 autoincrement 됨
cur.execute('''
    INSERT INTO CITY VALUES(NULL, '강북구')
''')

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'), (2, '동대문구'), (3, '강북구')]

In [20]:
# 가장 마지막에 추가된 row id
cur.lastrowid

3

### 1) excute : 단일 SQL만 실행 => Iterable한 객체 - 단일 데이터에 대한 Attributes

In [26]:
# 데이터 삽입2 : qmark 방식
cur.execute('INSERT INTO CITY(CNAME) VALUES(?)', (5, '강남구'))

In [27]:
# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'), (2, '동대문구'), (3, '강북구'), (4, '중구'), (5, '강남구')]

In [30]:
# 데이터 삽입3 : named 방식
cur.execute('INSERT INTO CITY(CNAME) VALUES(:key)', {'key':'강서구'})

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'),
 (2, '동대문구'),
 (3, '강북구'),
 (4, '중구'),
 (5, '강남구'),
 (6, '서초구'),
 (7, '중랑구'),
 (8, '강서구')]

In [31]:
cur.lastrowid

8

<br/>  

### 2) excutemany : 단일 SQL문을 여러 번 실행 => Iterable(Iterable)한 형태 - 여러 데이터에 대한 Attributes

In [32]:
# qmark 방식
cur.executemany('INSERT INTO CITY(CNAME) VALUES(?)', [('성북구1',),('성북구2',),('성북구3',)])

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'),
 (2, '동대문구'),
 (3, '강북구'),
 (4, '중구'),
 (5, '강남구'),
 (6, '서초구'),
 (7, '중랑구'),
 (8, '강서구'),
 (9, '성북구1'),
 (10, '성북구2'),
 (11, '성북구3')]

In [35]:
# named 방식
cur.executemany('INSERT INTO CITY(CNAME) VALUES(:key)', [('성북구4',),('성북구5',),('성북구6',)])

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'),
 (2, '동대문구'),
 (3, '강북구'),
 (4, '중구'),
 (5, '강남구'),
 (6, '서초구'),
 (7, '중랑구'),
 (8, '강서구'),
 (9, '성북구1'),
 (10, '성북구2'),
 (11, '성북구3'),
 (12, '성북구4'),
 (13, '성북구5'),
 (14, '성북구6')]

#### executemany 잘못 사용

밑의 경우처럼 사용시 성,북,구,7 이렇게 따로 저장하고 싶다면 VALUES(?,?,?,?) 이런식으로 코드를 작성해야함

In [42]:
# 잘못된 경우

cur.executemany('INSERT INTO CITY(CNAME) VALUES(?)', ('성북구7',))

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

OperationalError: 4 values for 1 columns

<br/> 

### 3) executescript : 비표준, 여러 SQL문을 실행 => 문(statement) 끝마다 ; 필요

In [43]:
cur.executescript('''
    INSERT INTO CITY(CNAME) VALUES('성동구7');
    INSERT INTO CITY(CNAME) VALUES('성동구8');
    INSERT INTO CITY(CNAME) VALUES('성동구9');
    INSERT INTO CITY(CNAME) VALUES('성동구10');
''')

# DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '강동구'),
 (2, '동대문구'),
 (3, '강북구'),
 (4, '중구'),
 (5, '강남구'),
 (6, '서초구'),
 (7, '중랑구'),
 (8, '강서구'),
 (9, '성북구1'),
 (10, '성북구2'),
 (11, '성북구3'),
 (12, '성북구4'),
 (13, '성북구5'),
 (14, '성북구6'),
 (15, '성동구7'),
 (16, '성동구8'),
 (17, '성동구9'),
 (18, '성동구10')]

In [45]:
# DB 연결 종료
con.close()

<BR/>

## 3. File로 DB 생성

In [46]:
con = sqlite3.connect('./practice.db')
cur = con.cursor()

In [51]:
# TABLE 생성
cur.executescript('''
    DROP TABLE IF EXISTS CITY;
    CREATE TABLE CITY(
        CNO   INTEGER PRIMARY KEY,
        CNAME TEXT
    );
    
    DROP TABLE IF EXISTS SUPPLIER;
    CREATE TABLE SUPPLIER(
        SNO   INTEGER PRIMARY KEY,
        SNAME TEXT,
        CNO   INTEGER NOT NULL
    );
    
    DROP TABLE IF EXISTS PART;
    CREATE TABLE PART(
        PNO   INTEGER PRIMARY KEY,
        PNAME TEXT
    );
    
    DROP TABLE IF EXISTS SELLS;
    CREATE TABLE SELLS(
        SNO   INTEGER,
        PNO   INTEGER,
        PRICE INTEGER
    );
''')

In [53]:
# CITY에 데이터 추가
cur.execute('INSERT INTO CITY(CNAME) VALUES(?)', ('성동구',))

# CITY 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구')]

In [54]:
cur.lastrowid

1

In [55]:
# CITY에 데이터 추가2
cur.executemany('INSERT INTO CITY(CNAME) VALUES(?)', [('성동구2',),('성동구3',),('성동구4',),('성동구5',)])

# CITY 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구'), (2, '성동구2'), (3, '성동구3'), (4, '성동구4'), (5, '성동구5')]

In [56]:
# DBMS에서 작업하던 것을 DB에 저장
con.commit()

In [57]:
# DBMS 종료
con.close()

In [58]:
# DBMS 다시 연결 후 저장한 DB 불러오기
con = sqlite3.connect('./practice.db')
cur = con.cursor()

# 저장된 DB 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구'), (2, '성동구2'), (3, '성동구3'), (4, '성동구4'), (5, '성동구5')]

In [62]:
# 새로운 데이터 추가
cityList = [('성동구',),('서초구',),('강북구',),('동대문구',)]

for city in cityList:
    cur.execute('SELECT * FROM CITY WHERE CNAME=?', city)
    if len(cur.fetchall()) == 0:
        cur.execute('INSERT INTO CITY(CNAME) VALUES(?)', city)
        
# CITY 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구'),
 (2, '성동구2'),
 (3, '성동구3'),
 (4, '성동구4'),
 (5, '성동구5'),
 (6, '서초구'),
 (7, '강북구'),
 (8, '동대문구')]

In [63]:
con.commit()

In [64]:
# SUPPLIER 데이터 추가
supplierList = [('1호점', 1),('2호점', 6),('3호점', 7),('4호점', 8)]

cur.executemany('INSERT INTO SUPPLIER(SNAME, CNO) VALUES(?, ?)', supplierList)

# SUPPLIER 확인
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1), (2, '2호점', 6), (3, '3호점', 7), (4, '4호점', 8)]

In [69]:
# SUPPLIER 데이터 추가2
supplierList = [('5호점', 1),('6호점', 1),('7호점', 8),('8호점', 8)]

cur.executemany('INSERT INTO SUPPLIER(SNAME, CNO) VALUES(?, ?)', supplierList)

# SUPPLIER 확인
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1),
 (2, '2호점', 6),
 (3, '3호점', 7),
 (4, '4호점', 8),
 (5, '5호점', 1),
 (6, '6호점', 1),
 (7, '7호점', 8),
 (8, '8호점', 8)]

In [70]:
con.commit()

In [76]:
# CNAME이 ?인 CNO 추출
cur.execute('SELECT CNO FROM CITY WHERE CNAME=?', ('서초구',))
row = cur.fetchone()
row

if row:
    CNO = row[0]
    
CNO

6

In [77]:
# 위에서 구한 CNO로 SUPPLIER 데이터 추가
cur.execute('INSERT INTO SUPPLIER(SNAME, CNO) VALUES(?,?)', ('9호점',CNO))

# SUPPLIER 확인
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1),
 (2, '2호점', 6),
 (3, '3호점', 7),
 (4, '4호점', 8),
 (5, '5호점', 1),
 (6, '6호점', 1),
 (7, '7호점', 8),
 (8, '8호점', 8),
 (9, '9호점', 6)]

In [109]:
# SUPPLIER에 SNAME은 10호점이고 CITY에서 CNAME이 서초구일 때 CNO값을 삽입하라
# LIMIT 0, 1 : 0번째 인덱스에서 1개만 가져와라
cur.execute('''INSERT INTO SUPPLIER(SNAME, CNO) VALUES(?,(
               SELECT CNO FROM CITY WHERE CNAME=? LIMIT 0,1
               ))
            ''', ('10호점', '서초구'))

# SUPPLIER 확인
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, '1호점', 1),
 (2, '2호점', 6),
 (3, '3호점', 7),
 (4, '4호점', 8),
 (5, '5호점', 1),
 (6, '6호점', 1),
 (7, '7호점', 8),
 (8, '8호점', 8),
 (9, '9호점', 6),
 (10, '10호점', 6)]

In [113]:
# 중간에 '대문'이 들어가는 CITY CNO 추출
cur.execute('SELECT * FROM CITY WHERE CNAME LIKE ?', ('%대문%',))
cur.fetchall()

[(8, '동대문구')]

In [114]:
con.commit()

### 1) CITY와 SUPPLIER CROSS JOIN

In [116]:
# cross join

cur.execute('''
    SELECT *
    FROM CITY, SUPPLIER 
    WHERE CITY.CNO = SUPPLIER.CNO
    ORDER BY CITY.CNAME
''')

cur.fetchall()

[(7, '강북구', 3, '3호점', 7),
 (8, '동대문구', 4, '4호점', 8),
 (8, '동대문구', 7, '7호점', 8),
 (8, '동대문구', 8, '8호점', 8),
 (6, '서초구', 2, '2호점', 6),
 (6, '서초구', 9, '9호점', 6),
 (6, '서초구', 10, '10호점', 6),
 (1, '성동구', 1, '1호점', 1),
 (1, '성동구', 5, '5호점', 1),
 (1, '성동구', 6, '6호점', 1)]

### 2) CITY 기준으로 SUPPLIER INNER JOIN

In [136]:
# 도시별 가맹점 inner join
cur.execute('''
    SELECT *
    FROM CITY
    INNER JOIN SUPPLIER
    WHERE CITY.CNO = SUPPLIER.CNO
    ORDER BY CITY.CNO
    
''')

cur.fetchall()

[(1, '성동구', 1, '1호점', 1),
 (1, '성동구', 5, '5호점', 1),
 (1, '성동구', 6, '6호점', 1),
 (6, '서초구', 2, '2호점', 6),
 (6, '서초구', 9, '9호점', 6),
 (6, '서초구', 10, '10호점', 6),
 (7, '강북구', 3, '3호점', 7),
 (8, '동대문구', 4, '4호점', 8),
 (8, '동대문구', 7, '7호점', 8),
 (8, '동대문구', 8, '8호점', 8)]

In [134]:
# 도시별 가맹점 갯수 inner join + GROUP BY
cur.execute('''
    SELECT CITY.CNAME, COUNT(SUPPLIER.SNAME)
    FROM CITY
    INNER JOIN SUPPLIER
    WHERE CITY.CNO = SUPPLIER.CNO
    GROUP BY CITY.CNO
    ORDER BY CITY.CNO
    
''')

cur.fetchall()

[('성동구', 3), ('서초구', 3), ('강북구', 1), ('동대문구', 3)]

### 3) CITY 기준으로 SUPPLIER LEFT JOIN

In [137]:
# CITY에 데이터 추가
cur.execute('INSERT INTO CITY(CNAME) VALUES(?)', ('중구',))

# CITY 확인
cur.execute('SELECT * FROM CITY')
cur.fetchall()

[(1, '성동구'),
 (2, '성동구2'),
 (3, '성동구3'),
 (4, '성동구4'),
 (5, '성동구5'),
 (6, '서초구'),
 (7, '강북구'),
 (8, '동대문구'),
 (9, '중구')]

In [140]:
# CITY를 기준으로 SUPPLIER LEFT JOIN

cur.execute('''
    SELECT *
    FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO = SUPPLIER.CNO
''')

cur.fetchall()

[(7, '강북구', 3, '3호점', 7),
 (8, '동대문구', 4, '4호점', 8),
 (6, '서초구', 10, '10호점', 6),
 (1, '성동구', 1, '1호점', 1),
 (2, '성동구2', None, None, None),
 (3, '성동구3', None, None, None),
 (4, '성동구4', None, None, None),
 (5, '성동구5', None, None, None),
 (9, '중구', None, None, None)]

In [144]:
# CITY를 기준으로 SUPPLIER LEFT JOIN + GROUP BY

cur.execute('''
    SELECT CITY.CNAME, COUNT(SUPPLIER.CNO)
    FROM CITY
    LEFT JOIN SUPPLIER
    ON CITY.CNO = SUPPLIER.CNO
    GROUP BY CITY.CNO
''')

cur.fetchall()

[('성동구', 3),
 ('성동구2', 0),
 ('성동구3', 0),
 ('성동구4', 0),
 ('성동구5', 0),
 ('서초구', 3),
 ('강북구', 1),
 ('동대문구', 3),
 ('중구', 0)]

In [149]:
con.commit()

In [150]:
# PART에 데이터 추가
partList = [('메뉴1',), ('메뉴2',), ('메뉴3',), ('메뉴4',), ('메뉴5',)]

cur.executemany('INSERT INTO PART(PNAME) VALUES(?)', partList)

# PART 확인
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, '메뉴1'), (2, '메뉴2'), (3, '메뉴3'), (4, '메뉴4'), (5, '메뉴5')]

In [176]:
# SALES 테이블의 모든 레코드 삭제
# cur.execute('DELETE FROM SELLS')

In [177]:
# SELLS 확인
# cur.execute('SELECT * FROM SELLS')
# cur.fetchall()

[]

In [178]:
# SELLS에 데이터 추가
sellList = [{'sno':1, 'pno':1, 'price':10},
            {'sno':2, 'pno':2, 'price':1},
            {'sno':3, 'pno':3, 'price':35},
            {'sno':4, 'pno':4, 'price':22},
            {'sno':5, 'pno':5, 'price':11},
            {'sno':6, 'pno':1, 'price':7},
            {'sno':7, 'pno':2, 'price':46},
            {'sno':8, 'pno':3, 'price':17},
            {'sno':9, 'pno':4, 'price':27},
            {'sno':10, 'pno':5, 'price':58},
            {'sno':1, 'pno':2, 'price':10},
             {'sno':1, 'pno':3, 'price':8},
             {'sno':1, 'pno':4, 'price':15},
             {'sno':1, 'pno':5, 'price':3},
             {'sno':2, 'pno':1, 'price':29},
             {'sno':2, 'pno':3, 'price':1},
             {'sno':2, 'pno':4, 'price':13},
             {'sno':2, 'pno':5, 'price':12},
             {'sno':3, 'pno':1, 'price':14},
             {'sno':3, 'pno':2, 'price':13},
             {'sno':3, 'pno':4, 'price':18},
             {'sno':3, 'pno':5, 'price':25},
             {'sno':4, 'pno':1, 'price':36},
             {'sno':4, 'pno':2, 'price':14},
             {'sno':4, 'pno':3, 'price':17},
             {'sno':4, 'pno':5, 'price':32},
             {'sno':5, 'pno':1, 'price':52},
             {'sno':5, 'pno':2, 'price':8},
             {'sno':5, 'pno':3, 'price':3},
             {'sno':5, 'pno':4, 'price':19}]

cur.executemany('''
    INSERT INTO SELLS VALUES(
    (SELECT SNO FROM SUPPLIER WHERE SNO LIKE :sno LIMIT 0,1),
    (SELECT PNO FROM PART WHERE PNO LIKE :pno LIMIT 0,1),
    :price
)''', sellList)

[(1, 1, 10),
 (2, 2, 1),
 (3, 3, 35),
 (4, 4, 22),
 (5, 5, 11),
 (6, 1, 7),
 (7, 2, 46),
 (8, 3, 17),
 (9, 4, 27),
 (10, 5, 58),
 (1, 2, 10),
 (1, 3, 8),
 (1, 4, 15),
 (1, 5, 3),
 (2, 1, 29),
 (2, 3, 1),
 (2, 4, 13),
 (2, 5, 12),
 (3, 1, 14),
 (3, 2, 13),
 (3, 4, 18),
 (3, 5, 25),
 (4, 1, 36),
 (4, 2, 14),
 (4, 3, 17),
 (4, 5, 32),
 (5, 1, 52),
 (5, 2, 8),
 (5, 3, 3),
 (5, 4, 19)]

In [179]:
# SELLS 확인
cur.execute('SELECT * FROM SELLS ORDER BY SNO')
cur.fetchall()

[(1, 1, 10),
 (1, 2, 10),
 (1, 3, 8),
 (1, 4, 15),
 (1, 5, 3),
 (2, 2, 1),
 (2, 1, 29),
 (2, 3, 1),
 (2, 4, 13),
 (2, 5, 12),
 (3, 3, 35),
 (3, 1, 14),
 (3, 2, 13),
 (3, 4, 18),
 (3, 5, 25),
 (4, 4, 22),
 (4, 1, 36),
 (4, 2, 14),
 (4, 3, 17),
 (4, 5, 32),
 (5, 5, 11),
 (5, 1, 52),
 (5, 2, 8),
 (5, 3, 3),
 (5, 4, 19),
 (6, 1, 7),
 (7, 2, 46),
 (8, 3, 17),
 (9, 4, 27),
 (10, 5, 58)]

In [180]:
con.commit()

### 4) 가맹점 별 매출

In [4]:
cur.execute('''
    SELECT SUM(SELLS.PRICE), SUPPLIER.SNAME, CITY.CNAME
    FROM SELLS
    INNER JOIN SUPPLIER
    ON SUPPLIER.SNO = SELLS.SNO
    INNER JOIN PART
    ON PART.PNO = SELLS.PNO
    INNER JOIN CITY
    ON CITY.CNO = SUPPLIER.CNO
    GROUP BY CITY.CNAME,SUPPLIER.SNAME
    ORDER BY SUM(SELLS.PRICE) DESC
''')

cur.fetchall()

[(121, '4호점', '동대문구'),
 (105, '3호점', '강북구'),
 (93, '5호점', '성동구'),
 (58, '10호점', '서초구'),
 (56, '2호점', '서초구'),
 (46, '7호점', '동대문구'),
 (46, '1호점', '성동구'),
 (27, '9호점', '서초구'),
 (17, '8호점', '동대문구'),
 (7, '6호점', '성동구')]

<br/>

# 4. 함수 형태로 DB 관리 (ver.간단)

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect('./practice2')
cur = con.cursor()

In [3]:
# 테이블 생성
cur.executescript('''
    DROP TABLE IF EXISTS artist;
    CREATE TABLE artist(
        arpk   INTEGER PRIMARY KEY,
        arname TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS album;
    CREATE TABLE album(
        alpk   INTEGER PRIMARY KEY,
        alname TEXT NOT NULL,
        arpk   INTEGER NOT NULL
    );
    
    DROP TABLE IF EXISTS genre;
    CREATE TABLE genre(
        gpk   INTEGER PRIMARY KEY,
        gname TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS track;
    CREATE TABLE track(
        tpk    INTEGER PRIMARY KEY,
        tname  TEXT NOT NULL DEFAULT '무제',
        length INTEGER DEFAULT 0,
        rating INTEGER DEFAULT 0,
        count  INTEGER DEFAULT 0,
        alfk   INTEGER NOT NULL,
        gfk    INTEGER NOT NULL
    );
''')

In [4]:
# 데이터 추가
def addArtist(name, flag=0):
    if flag == 0:
        v = name
    elif flag == 1:
        v = '%'+name
    elif flag == 2:
        v = name + '%'
    else:
        v = '%' + name + '%'
        
    cur.execute('SELECT arpk FROM artist WHERE arname LIKE ?', (v,))
    rst = cur.fetchone()  # fetchone하면 SELECT 결과가 없는 경우 None 반환
    
    # 데이터에 없는 이름인 경우 삽입
    if rst is None:
        cur.execute('INSERT INTO artist(arname) VALUES(?)', (v,))
        result = cur.lastrowid
        
    # 데이터에 없는 이름인 경우 삽입 안함
    else:
        result = rst[0]
        
    return result

In [5]:
name = ['아이브', '박효신', '성시경', '지코', '세븐틴', '김조한', '아이유', '다이나믹듀오']

for i in range(len(name)):
    addArtist(name[i], 0)

In [6]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '아이브'),
 (2, '박효신'),
 (3, '성시경'),
 (4, '지코'),
 (5, '세븐틴'),
 (6, '김조한'),
 (7, '아이유'),
 (8, '다이나믹듀오')]

In [32]:
# cur.execute('DELETE FROM artist')

In [7]:
# 데이터 변경
def modArtist(name, rename):
    arpk = addArtist(name)
    cur.execute('UPDATE artist SET arname=? WHERE arpk=?', (rename, arpk))
    
    return cur.rowcount

In [8]:
modArtist('다이나믹듀오', '다듀')

1

In [9]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '아이브'),
 (2, '박효신'),
 (3, '성시경'),
 (4, '지코'),
 (5, '세븐틴'),
 (6, '김조한'),
 (7, '아이유'),
 (8, '다듀')]

In [14]:
# 데이터 삭제
def delArtist(name):
    arpk = addArtist(name)
#     print(arpk)
    
    cur.execute('DELETE FROM artist WHERE arname=?', (name,))
    return cur.rowcount

In [15]:
delArtist('아이브')

1

In [16]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(2, '박효신'),
 (3, '성시경'),
 (4, '지코'),
 (5, '세븐틴'),
 (6, '김조한'),
 (7, '아이유'),
 (8, '다듀')]

In [17]:
con.close()

<br/> 

# 5. 함수 형태로 DB 관리 (ver.모든 table 동시 사용)

In [18]:
import sqlite3

In [19]:
con = sqlite3.connect('./practice3')
cur = con.cursor()

In [21]:
# table 생성
cur.executescript('''
    DROP TABLE IF EXISTS artist;
    CREATE TABLE artist(
        arpk    INTEGER PRIMARY KEY,
        arname  TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS album;
    CREATE TABLE album(
        alpk    INTEGER PRIMARY KEY,
        alname  TEXT NOT NULL,
        arpk    INTEGER NOT NULL
    );
    
    DROP TABLE IF EXISTS genre;
    CREATE TABLE genre(
        gpk    INTEGER PRIMARY KEY,
        gname  TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS track;
    CREATE TABLE track(
        tpk    INTEGER PRIMARY KEY,
        tname  TEXT NOT NULL DEFAULT '무제',
        length INTEGER DEFAULT 0,
        rating INTEGER DEFAULT 0,
        count  INTEGER DEFAULT 0,
        alpk   INTEGER NOT NULL,
        gpk    INTEGER NOT NULL
    );
''')

In [130]:
# 테이블별 데이터 추가
def addRow(tablename, columns, values):
    if len(columns) != len(values):
        return False

    where = [columns[i] +'=?' for i in range(len(columns))] # columns[0]=?, columns[1]=? ,.... 형태
    c = ' and '.join(where)  # columns[0] and columns[1] and ... 형태
    d = ','.join(columns)       # columns[0], columns[1], ... 형태
    q = ','.join(['?' for i in range(len(values))]) # ?,?,?,... 형태

    # 추가하려는 데이터가 이미 존재하는지 확인
    cur.execute('SELECT * FROM ' +tablename+ ' WHERE ' +c, values)
    rst = cur.fetchone()

    
    # 데이터가 존재하지 않다면 삽입 후 pk값 반환
    if rst is None:
        cur.execute('INSERT INTO ' +tablename+ '(' +d+ ') VALUES(' +q+ ')', values)
        result = cur.lastrowid
        
    # 데이터가 이미 존재한다면 pk값 반환
    else:
        result = rst[0]
        
    return result

In [161]:
# artist에 데이터 삽입
name = ['아이브', '아이유', '다이나믹 듀오', '박효신', '제이레빗']

[addRow('artist', ['arname'], [i]) for i in name]

[1, 2, 3, 4, 5]

In [227]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '아이브'), (2, '아이유'), (3, '다이나믹 듀오'), (4, '박효신'), (5, '제이레빗')]

In [ ]:
# cur.execute('DELETE FROM artist')

In [165]:
# genre에 데이터 삽입
name = ['댄스', '발라드', '락', '힙합', '알앤비', 'ost', '재즈']

[addRow('genre', ['gname'], [i]) for i in name]

[1, 2, 3, 4, 5, 6, 7]

In [166]:
cur.execute('SELECT * FROM genre')
cur.fetchall()

[(1, '댄스'), (2, '발라드'), (3, '락'), (4, '힙합'), (5, '알앤비'), (6, 'ost'), (7, '재즈')]

In [ ]:
# cur.execute('DELETE FROM genre')

In [243]:
# album에 데이터 삽입
name = ['아이브', '아이유', '다이나믹 듀오', '박효신', '제이레빗']
alname = [['I\'ve IVE', 'after like', 'LOVE DIVE'], ['조각집', 'strawberry moon', 'modern times', '스무살의 봄'],
         ['soon','나 오늘','blue','마음의 소리'], ['goodbye','야생화','happy together'],
         ['joy to the world', '슬기로운 의사생활', '연애의 발견']]

arpk = [addRow('artist', ['arname'], [i]) for i in name]

alnameSplit = [j for i in alname for j in i]


# Comprehension 방식
[addRow('album', ['alname', 'arpk'], [j, i+1]) for i in range(len(name)) for j in alname[i]]


# For문 방식
# for i in range(len(name)):
#     for j in alname[i]:
#         addRow('album', ['alname', 'arpk'], [j, i+1])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [244]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, "I've IVE", 1),
 (2, 'after like', 1),
 (3, 'LOVE DIVE', 1),
 (4, '조각집', 2),
 (5, 'strawberry moon', 2),
 (6, 'modern times', 2),
 (7, '스무살의 봄', 2),
 (8, 'soon', 3),
 (9, '나 오늘', 3),
 (10, 'blue', 3),
 (11, '마음의 소리', 3),
 (12, 'goodbye', 4),
 (13, '야생화', 4),
 (14, 'happy together', 4),
 (15, 'joy to the world', 5),
 (16, '슬기로운 의사생활', 5),
 (17, '연애의 발견', 5)]

In [236]:
# cur.execute('DELETE FROM album')

In [213]:
# track에 데이터 삽입
data = [{'name': 'I AM', 'gpk': 1, 'alpk': 1},
        {'name': 'Kitsch', 'gpk': 1, 'alpk': 1},
        {'name': 'After LIKE', 'gpk': 1, 'alpk': 2},
        {'name': 'LOVE DIVE', 'gpk': 1, 'alpk': 3},
        {'name': 'ROYAL', 'gpk': 1, 'alpk': 3},
        {'name': '드라마', 'gpk': 2, 'alpk': 4},
        {'name': '겨울잠', 'gpk': 2, 'alpk': 4},
        {'name': 'strawberry moon', 'gpk': 3, 'alpk': 5},
        {'name': '을의 연애', 'gpk': 1, 'alpk': 6},
        {'name': '분홍신', 'gpk': 1, 'alpk': 6},
        {'name': 'Modern Times', 'gpk': 1, 'alpk': 6},
        {'name': '우울시계', 'gpk': 2, 'alpk': 6},
        {'name': '하루 끝', 'gpk': 2, 'alpk': 7},
        {'name': 'soon', 'gpk': 4, 'alpk': 8},
        {'name': '나 오늘', 'gpk': 5, 'alpk': 9},
        {'name': 'blue', 'gpk': 4, 'alpk': 10},
        {'name': '좋겠다', 'gpk': 6, 'alpk': 11},
        {'name': 'goodbye', 'gpk': 2, 'alpk': 12},
        {'name': '야생화', 'gpk': 2, 'alpk': 13},
        {'name': 'happy together', 'gpk': 2, 'alpk': 14},
        {'name': 'joy to the wolrd', 'gpk': 7, 'alpk': 15},
        {'name': '넌 언제나', 'gpk': 6, 'alpk': 16},
        {'name': '사랑일까?', 'gpk': 6, 'alpk': 17}
       ]

# 방법 1
[addRow('track', ['tname', 'gpk', 'alpk'], [i['name'], i['gpk'], i['alpk']]) for i in data]

# 방법 2
# for i in data:
#     addRow('track', ['tname', 'gpk', 'alpk'], [i['name'], i['gpk'], i['alpk']])

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [214]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, 'I AM', 0, 0, 0, 1, 1),
 (2, 'Kitsch', 0, 0, 0, 1, 1),
 (3, 'After LIKE', 0, 0, 0, 2, 1),
 (4, 'LOVE DIVE', 0, 0, 0, 3, 1),
 (5, 'ROYAL', 0, 0, 0, 3, 1),
 (6, '드라마', 0, 0, 0, 4, 2),
 (7, '겨울잠', 0, 0, 0, 4, 2),
 (8, 'strawberry moon', 0, 0, 0, 5, 3),
 (9, '을의 연애', 0, 0, 0, 6, 1),
 (10, '분홍신', 0, 0, 0, 6, 1),
 (11, 'Modern Times', 0, 0, 0, 6, 1),
 (12, '우울시계', 0, 0, 0, 6, 2),
 (13, '하루 끝', 0, 0, 0, 7, 2),
 (14, 'soon', 0, 0, 0, 8, 4),
 (15, '나 오늘', 0, 0, 0, 9, 5),
 (16, 'blue', 0, 0, 0, 10, 4),
 (17, '좋겠다', 0, 0, 0, 11, 6),
 (18, 'goodbye', 0, 0, 0, 12, 2),
 (19, '야생화', 0, 0, 0, 13, 2),
 (20, 'happy together', 0, 0, 0, 14, 2),
 (21, 'joy to the wolrd', 0, 0, 0, 15, 7),
 (22, '넌 언제나', 0, 0, 0, 16, 6),
 (23, '사랑일까?', 0, 0, 0, 17, 6)]

In [211]:
# cur.execute('DELETE FROM track')

In [215]:
con.commit()

<br/> 

## Join

In [246]:
# 가수 기준일 때 가수의 앨범 불러오기
cur.execute('''
    SELECT artist.arname, album.alname
    FROM artist
    LEFT JOIN album
    ON artist.arpk=album.arpk
''')

cur.fetchall()

[('아이브', "I've IVE"),
 ('아이브', 'LOVE DIVE'),
 ('아이브', 'after like'),
 ('아이유', 'modern times'),
 ('아이유', 'strawberry moon'),
 ('아이유', '스무살의 봄'),
 ('아이유', '조각집'),
 ('다이나믹 듀오', 'blue'),
 ('다이나믹 듀오', 'soon'),
 ('다이나믹 듀오', '나 오늘'),
 ('다이나믹 듀오', '마음의 소리'),
 ('박효신', 'goodbye'),
 ('박효신', 'happy together'),
 ('박효신', '야생화'),
 ('제이레빗', 'joy to the world'),
 ('제이레빗', '슬기로운 의사생활'),
 ('제이레빗', '연애의 발견')]

In [248]:
# 가수 기준일 때 가수의 앨범 갯수 불러오기
cur.execute('''
    SELECT artist.arname, count(album.arpk)
    FROM artist
    LEFT JOIN album
    ON artist.arpk=album.arpk
    GROUP BY artist.arpk
''')

cur.fetchall()

[('아이브', 3), ('아이유', 4), ('다이나믹 듀오', 4), ('박효신', 3), ('제이레빗', 3)]

In [251]:
# 장르 별 노래 목록
cur.execute('''
    SELECT genre.gname, track.tname
    FROM genre
    LEFT JOIN track
    ON track.gpk = genre.gpk
''')
cur.fetchall()

[('댄스', 'After LIKE'),
 ('댄스', 'I AM'),
 ('댄스', 'Kitsch'),
 ('댄스', 'LOVE DIVE'),
 ('댄스', 'Modern Times'),
 ('댄스', 'ROYAL'),
 ('댄스', '분홍신'),
 ('댄스', '을의 연애'),
 ('발라드', 'goodbye'),
 ('발라드', 'happy together'),
 ('발라드', '겨울잠'),
 ('발라드', '드라마'),
 ('발라드', '야생화'),
 ('발라드', '우울시계'),
 ('발라드', '하루 끝'),
 ('락', 'strawberry moon'),
 ('힙합', 'blue'),
 ('힙합', 'soon'),
 ('알앤비', '나 오늘'),
 ('ost', '넌 언제나'),
 ('ost', '사랑일까?'),
 ('ost', '좋겠다'),
 ('재즈', 'joy to the wolrd')]

In [256]:
# 장르 별 노래 갯수
cur.execute('''
    SELECT genre.gname, count(track.gpk)
    FROM genre
    LEFT JOIN track
    ON track.gpk = genre.gpk
    GROUP BY genre.gpk
    ORDER BY count(track.gpk) DESC, genre.gname
''')
cur.fetchall()

[('댄스', 8), ('발라드', 7), ('ost', 3), ('힙합', 2), ('락', 1), ('알앤비', 1), ('재즈', 1)]

In [259]:
# 전체 목록
cur.execute('''
    SELECT artist.arname, album.alname, genre.gname, track.tname
    FROM track
    INNER JOIN genre ON track.gpk=genre.gpk
    INNER JOIN album ON track.alpk=album.alpk
    INNER JOIN artist ON album.arpk=artist.arpk
''')
cur.fetchall()

[('아이브', "I've IVE", '댄스', 'I AM'),
 ('아이브', "I've IVE", '댄스', 'Kitsch'),
 ('아이브', 'after like', '댄스', 'After LIKE'),
 ('아이브', 'LOVE DIVE', '댄스', 'LOVE DIVE'),
 ('아이브', 'LOVE DIVE', '댄스', 'ROYAL'),
 ('아이유', '조각집', '발라드', '드라마'),
 ('아이유', '조각집', '발라드', '겨울잠'),
 ('아이유', 'strawberry moon', '락', 'strawberry moon'),
 ('아이유', 'modern times', '댄스', '을의 연애'),
 ('아이유', 'modern times', '댄스', '분홍신'),
 ('아이유', 'modern times', '댄스', 'Modern Times'),
 ('아이유', 'modern times', '발라드', '우울시계'),
 ('아이유', '스무살의 봄', '발라드', '하루 끝'),
 ('다이나믹 듀오', 'soon', '힙합', 'soon'),
 ('다이나믹 듀오', '나 오늘', '알앤비', '나 오늘'),
 ('다이나믹 듀오', 'blue', '힙합', 'blue'),
 ('다이나믹 듀오', '마음의 소리', 'ost', '좋겠다'),
 ('박효신', 'goodbye', '발라드', 'goodbye'),
 ('박효신', '야생화', '발라드', '야생화'),
 ('박효신', 'happy together', '발라드', 'happy together'),
 ('제이레빗', 'joy to the world', '재즈', 'joy to the wolrd'),
 ('제이레빗', '슬기로운 의사생활', 'ost', '넌 언제나'),
 ('제이레빗', '연애의 발견', 'ost', '사랑일까?')]

In [267]:
# 장르가 발라드일 때 노래가 짧은 것 순서대로
cur.execute('''
    SELECT artist.arname, album.alname, track.tname, track.length
    FROM track
    INNER JOIN genre ON genre.gpk=track.gpk
    INNER JOIN album ON album.alpk=track.alpk
    INNER JOIN artist ON artist.arpk=album.arpk
    WHERE genre.gname='발라드'
    ORDER BY track.length
''')

cur.fetchall()

[('아이유', '조각집', '드라마', 0),
 ('아이유', '조각집', '겨울잠', 0),
 ('아이유', 'modern times', '우울시계', 0),
 ('아이유', '스무살의 봄', '하루 끝', 0),
 ('박효신', 'goodbye', 'goodbye', 0),
 ('박효신', '야생화', '야생화', 0),
 ('박효신', 'happy together', 'happy together', 0)]

In [265]:
# 발라드 노래 갯수
cur.execute('''
    SELECT genre.gname, count(genre.gpk) 
    FROM track
    INNER JOIN genre ON genre.gpk=track.gpk
    WHERE genre.gname = '발라드'
''')

cur.fetchall()

[('발라드', 7)]